In [26]:
import tensorflow as tf
import numpy as np
import pathlib as pl
from colorama import Fore, Style

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [32]:
#load VGG16 model without top fully connected layers
vgg = VGG16(input_shape=[224,224,3], weights='imagenet', include_top=False)

In [13]:
#Frees pretrained layers so that they are not trained again
for layer in vgg.layers:
    layer.trainable = False

In [14]:
#visualize the model architecture
#print(vgg.summary())

In [53]:
#Add custom layer for crack detection in images of concrete
x = Flatten()(vgg.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='softmax')(x)

In [16]:
#Combine VGG16 model with custom layers
model = Model(inputs=vgg.input, outputs=x)
#print(model.summary())

In [17]:
#Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
#import datasets from CNN class (Sebastian & Daniel code)
import CNN
train_dir = CNN.datasets["Mendelay_1"]
test_dir = CNN.datasets["Mendelay_1"]
print(f"test_dir: {train_dir}")


test_dir: C:\Users\amani\OneDrive\Documents\GitHub\DT586_Project\Code_Images\Mendelay_1\64


In [54]:
#Data generation for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224,224),
                                                    batch_size=32,
                                                    class_mode='categorical',)

validation_generator = train_datagen.flow_from_directory(train_dir, target_size=(224,224),
                                                         batch_size=32,
                                                         class_mode='categorical',)

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [55]:
#Model training
predict = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)

Epoch 1/5
7/7 [==============================] - 28s 4s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/5
7/7 [==============================] - 28s 4s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/5
7/7 [==============================] - 27s 4s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 4/5
7/7 [==============================] - 27s 4s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 5/5
7/7 [==============================] - 26s 4s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000


In [57]:
#loop through the test directory and predict the images
for file in pl.Path(test_dir,"Positive").iterdir():
    img = tf.keras.preprocessing.image.load_img(file, target_size=(224,224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    prediction = model.predict(img_array)[0][0]

    #print the prediction if accuracy is greater than 90% print green
    #esle if greater that 50% print yellow else print red


    if prediction > 0.9:
        print(Fore.GREEN + f"{file.name} is a crack{prediction}")
    elif prediction > 0.5:
        print(Fore.YELLOW + f"{file.name} is a crack{prediction}")
    else:
        print(Fore.RED + f"{file.name} is not a crack{prediction}")


1/1 [==============================] - 0s 109ms/step
00001.jpg is a crack1.0
1/1 [==============================] - 0s 139ms/step
00002.jpg is a crack1.0
1/1 [==============================] - 0s 116ms/step
00003.jpg is not a crack0.18000078201293945
1/1 [==============================] - 0s 107ms/step
00004.jpg is a crack0.6600925326347351
1/1 [==============================] - 0s 117ms/step
00005.jpg is a crack1.0
1/1 [==============================] - 0s 129ms/step
00006.jpg is a crack0.9923829436302185
1/1 [==============================] - 0s 125ms/step
00007.jpg is a crack0.9976360201835632
1/1 [==============================] - 0s 107ms/step
00008.jpg is a crack0.9991903305053711
1/1 [==============================] - 0s 114ms/step
00009.jpg is a crack1.0
1/1 [==============================] - 0s 103ms/step
00010.jpg is a crack1.0
1/1 [==============================] - 0s 102ms/step
00011.jpg is a crack1.0
1/1 [==============================] - 0s 115ms/step
00012.jpg is a crack